In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)


In [ ]:

import os
if not os.path.isdir('saved_models_mnist_{}'.format('sngan')):
    os.mkdir('saved_models_mnist_{}'.format('sngan'))
f = open('saved_models_mnist_{}/log_collapse1.txt'.format('sngan'), mode='w')
import os
import tensorflow as tf
import torch.utils.data as Data
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10,fashion_mnist,mnist
import os
from scipy import misc
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose, Conv2D
from keras.optimizers import Adam
import os
from keras.layers.convolutional import _Conv
from keras.legacy import interfaces
from keras.engine import InputSpec
import cv2
import keras.backend.tensorflow_backend as KTF

from scipy import misc
def set_gpu_config(device = "0",fraction=0.25):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = fraction
    config.gpu_options.visible_device_list = device
    KTF.set_session(tf.Session(config=config))


def predict_images(file_name, generator, noise_size, n = 10, size = 32):

    image = generator.predict(np.random.normal(size=(n*n, ) + noise_size))

    image = np.reshape(image, (n, n, size, size, 3))
    image = np.transpose(image, (0, 2, 1, 3, 4))
    image = np.reshape(image, (n*size, n*size, 3))

    image = 255 * (image + 1) / 2
    image = image.astype("uint8")
    misc.imsave(file_name, image)
def build_generator(input_shape):
    model = Sequential()

    model.add(Conv2DTranspose(512,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(256,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(64,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(1,(3,3),padding="same",activation="tanh"))
    model.summary()
    return model


def build_discriminator(input_shape):
    model = Sequential()

    model.add(SNConv2D(64,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(256,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(512,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(SNConv2D(1,(3,3),padding="same"))
    model.add(GlobalAveragePooling2D())
    model.summary()

    return model

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)
    fake_image = generator(noise)

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    d_loss = K.mean(K.maximum(0., 1 - pred_real)) + K.mean(K.maximum(0., 1 + pred_fake))
    g_loss = -K.mean(pred_fake)

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss], d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_training_updates)

    return d_train,g_train

class SNConv2D(_Conv):
    @interfaces.legacy_conv2d_support
    def __init__(self, filters,
                 kernel_size,
                 strides=(1, 1),
                 padding='valid',
                 data_format=None,
                 dilation_rate=(1, 1),
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):

        super(SNConv2D, self).__init__(
            rank=2,
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            **kwargs)

        self.input_spec = InputSpec(ndim=4)
        self.Ip = 1
        self.u = self.add_weight(
            name='W_u',
            shape=(1,filters),
            initializer='random_uniform',
            trainable=False
        )

    def call(self, inputs):
        outputs = K.conv2d(
            inputs,
            self.W_bar(),
            strides=self.strides,
            padding=self.padding,
            data_format=self.data_format,
            dilation_rate=self.dilation_rate)

        if self.use_bias:
            outputs = K.bias_add(
                outputs,
                self.bias,
                data_format=self.data_format)

        if self.activation is not None:
            return self.activation(outputs)
        return outputs


    def get_config(self):
        config = super(SNConv2D, self).get_config()
        config.pop('rank')
        return config

    def W_bar(self):
        # Spectrally Normalized Weight
        W_mat = K.permute_dimensions(self.kernel, (3, 2, 0, 1)) # (h, w, i, o) => (o, i, h, w)
        W_mat = K.reshape(W_mat,[K.shape(W_mat)[0], -1]) # (o, i * h * w)

        if not self.Ip >= 1:
            raise ValueError("The number of power iterations should be positive integer")

        _u = self.u
        _v = None

        for _ in range(self.Ip):
            _v = _l2normalize(K.dot(_u, W_mat))
            _u = _l2normalize(K.dot(_v, K.transpose(W_mat)))

        sigma = K.sum(K.dot(_u,W_mat)*_v)

        K.update(self.u,K.in_train_phase(_u, self.u))
        return self.kernel / sigma

def _l2normalize(x):
    return x / K.sqrt(K.sum(K.square(x)) + K.epsilon())
set_gpu_config("0",0.5)

epochs = 50
image_size = (32,32,1)
noise_size = (2,2,32)
batch_size = 64
sample_size=10
size=32
sample_interval=200
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train=[cv2.resize(i,(32,32)) for i in x_train]

x_train=np.array(x_train)
x_test=np.array(x_test)
x_train=np.expand_dims(x_train,axis=3)
x_test=np.expand_dims(x_test,axis=3)
num_of_data = x_train.shape[0]
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train = (x_train/255)*2-1
x_test = (x_test/255)*2-1
y_train = keras.utils.to_categorical(y_train,10)
# y_test = keras.utils.to_categorical(y_test,10)
x = []
y = np.zeros((31, 1), dtype=np.int)
y = list(y)
for i in range(31):
    y[i] = []
generator = build_generator(noise_size)
discriminator = build_discriminator(image_size)
d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

nb_batches = int(x_train.shape[0] / batch_size)
global_step = 0
for epoch in range(epochs):
    for index in range(nb_batches):
        global_step += 1
        real_images = x_train[index * batch_size:(index + 1) * batch_size]
        d_loss, = d_train([real_images, 1])
        g_loss, = g_train([real_images, 1])
        print("[{0}/{1}] [{2}_{3}] d_loss: {4:.4}, g_loss: {5:.4}".format(epoch, epochs, epoch, global_step, d_loss,
                                                                              g_loss))
        sampleSize=10000
        if global_step % sample_interval == 0:
            image = generator.predict(np.random.normal(size=(sampleSize,) + noise_size))
            image = 255 * (image + 1) / 2
            image = [cv2.resize(i, (28, 28)) for i in image]
            image = np.expand_dims(image, axis=3)
            image = np.array(image)
            labels = np.squeeze(y_test[:sampleSize])
            labels = np.squeeze(labels)
            dis_real = cal_distance_image_real(x_test[:sampleSize], labels)
            dis_fake = cal_distance_image_fake(image )
            dis_cha = dis_real - dis_fake
            print('##############')
            # print(dis_real)
            # print(dis_fake)
            print(dis_cha)
            print('##########')
            f.writelines('\n')
            f.writelines('epoch:' + str(epoch))
            f.writelines('\n')
            f.writelines('紧度')
            f.writelines('\n')
            f.writelines(' %.8f ' % (i) for i in dis_cha)
            f.writelines('\n')



Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_1 (Conv2DTr (None, 4, 4, 512)         147968    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 8, 8, 256)         1179904   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 16, 16, 128)       295040    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 32, 32, 64)        73792     
__________

[0/50] [0_123] d_loss: 0.0969, g_loss: 2.652
[0/50] [0_124] d_loss: 0.01835, g_loss: 2.26
[0/50] [0_125] d_loss: 0.0181, g_loss: 1.867
[0/50] [0_126] d_loss: 0.0, g_loss: 1.54
[0/50] [0_127] d_loss: 0.008368, g_loss: 1.96
[0/50] [0_128] d_loss: 0.008609, g_loss: 1.428
[0/50] [0_129] d_loss: 0.0196, g_loss: 2.567
[0/50] [0_130] d_loss: 0.05788, g_loss: 1.827
[0/50] [0_131] d_loss: 0.0, g_loss: 1.604
[0/50] [0_132] d_loss: 0.06352, g_loss: 1.033
[0/50] [0_133] d_loss: 0.2127, g_loss: 6.879
[0/50] [0_134] d_loss: 1.453, g_loss: 1.141
[0/50] [0_135] d_loss: 0.07714, g_loss: 2.206
[0/50] [0_136] d_loss: 0.02839, g_loss: 1.782
[0/50] [0_137] d_loss: 0.02643, g_loss: 2.207
[0/50] [0_138] d_loss: 0.05383, g_loss: 3.357
[0/50] [0_139] d_loss: 0.1013, g_loss: 0.9702
[0/50] [0_140] d_loss: 3.454, g_loss: 4.628
[0/50] [0_141] d_loss: 2.856, g_loss: 1.09
[0/50] [0_142] d_loss: 0.2884, g_loss: 1.2
[0/50] [0_143] d_loss: 0.2442, g_loss: 1.189
[0/50] [0_144] d_loss: 0.1038, g_loss: 1.685
[0/50] [0_145

[0/50] [0_296] d_loss: 0.4857, g_loss: 0.8517
[0/50] [0_297] d_loss: 0.7769, g_loss: 3.928
[0/50] [0_298] d_loss: 1.077, g_loss: 1.145
[0/50] [0_299] d_loss: 0.4311, g_loss: 1.951
[0/50] [0_300] d_loss: 0.3942, g_loss: 1.23
[0/50] [0_301] d_loss: 0.477, g_loss: 3.338
[0/50] [0_302] d_loss: 0.7303, g_loss: 1.058
[0/50] [0_303] d_loss: 0.4814, g_loss: 3.461
[0/50] [0_304] d_loss: 0.7921, g_loss: 0.8037
[0/50] [0_305] d_loss: 0.6642, g_loss: 2.487
[0/50] [0_306] d_loss: 0.452, g_loss: 1.224
[0/50] [0_307] d_loss: 0.4936, g_loss: 2.783
[0/50] [0_308] d_loss: 0.6943, g_loss: 0.6342
[0/50] [0_309] d_loss: 0.7766, g_loss: 3.076
[0/50] [0_310] d_loss: 0.762, g_loss: 1.156
[0/50] [0_311] d_loss: 0.4329, g_loss: 1.914
[0/50] [0_312] d_loss: 0.4677, g_loss: 0.3869
[0/50] [0_313] d_loss: 1.098, g_loss: 3.03
[0/50] [0_314] d_loss: 1.052, g_loss: 0.9836
[0/50] [0_315] d_loss: 0.6723, g_loss: 1.7
[0/50] [0_316] d_loss: 0.5104, g_loss: 1.602
[0/50] [0_317] d_loss: 0.3611, g_loss: 1.77
[0/50] [0_318] d

[0/50] [0_480] d_loss: 0.6654, g_loss: 1.216
[0/50] [0_481] d_loss: 0.539, g_loss: 2.35
[0/50] [0_482] d_loss: 0.3657, g_loss: 0.9139
[0/50] [0_483] d_loss: 0.6933, g_loss: 4.632
[0/50] [0_484] d_loss: 1.233, g_loss: 0.6233
[0/50] [0_485] d_loss: 0.9206, g_loss: 1.299
[0/50] [0_486] d_loss: 0.8664, g_loss: 2.417
[0/50] [0_487] d_loss: 0.76, g_loss: -0.2076
[0/50] [0_488] d_loss: 1.488, g_loss: 1.128
[0/50] [0_489] d_loss: 0.947, g_loss: 1.087
[0/50] [0_490] d_loss: 0.4976, g_loss: 1.229
[0/50] [0_491] d_loss: 0.5627, g_loss: 1.033
[0/50] [0_492] d_loss: 0.4198, g_loss: 2.044
[0/50] [0_493] d_loss: 0.6298, g_loss: 1.045
[0/50] [0_494] d_loss: 0.4729, g_loss: 2.253
[0/50] [0_495] d_loss: 0.7123, g_loss: 1.492
[0/50] [0_496] d_loss: 0.5635, g_loss: 2.338
[0/50] [0_497] d_loss: 0.4916, g_loss: 0.9149
[0/50] [0_498] d_loss: 0.5197, g_loss: 3.745
[0/50] [0_499] d_loss: 1.05, g_loss: 1.042
[0/50] [0_500] d_loss: 0.5904, g_loss: 2.141
[0/50] [0_501] d_loss: 0.6179, g_loss: 0.7225
[0/50] [0_502

...

[39/50] [39_37260] d_loss: 0.08171, g_loss: 2.593
[39/50] [39_37261] d_loss: 0.2298, g_loss: 1.198
[39/50] [39_37262] d_loss: 0.1005, g_loss: 2.466
[39/50] [39_37263] d_loss: 0.008672, g_loss: 2.951
[39/50] [39_37264] d_loss: 0.01068, g_loss: 3.172
[39/50] [39_37265] d_loss: 0.0, g_loss: 3.344
[39/50] [39_37266] d_loss: 0.1055, g_loss: 1.525
[39/50] [39_37267] d_loss: 0.0936, g_loss: 2.281
[39/50] [39_37268] d_loss: 0.08262, g_loss: 1.782
[39/50] [39_37269] d_loss: 0.09455, g_loss: 1.946
[39/50] [39_37270] d_loss: 0.03799, g_loss: 1.799
[39/50] [39_37271] d_loss: 0.00388, g_loss: 1.951
[39/50] [39_37272] d_loss: 0.08822, g_loss: 2.551
[39/50] [39_37273] d_loss: 0.03898, g_loss: 1.913
[39/50] [39_37274] d_loss: 0.04476, g_loss: 1.889
[39/50] [39_37275] d_loss: 0.02928, g_loss: 1.562
[39/50] [39_37276] d_loss: 0.04046, g_loss: 2.147
[39/50] [39_37277] d_loss: 0.03888, g_loss: 1.97
[39/50] [39_37278] d_loss: 0.04776, g_loss: 2.028
[39/50] [39_37279] d_loss: 0.01574, g_loss: 2.32
[39/50] [

[39/50] [39_37430] d_loss: 0.04748, g_loss: 2.586
[39/50] [39_37431] d_loss: 0.0, g_loss: 2.541
[39/50] [39_37432] d_loss: 0.02819, g_loss: 2.55
[39/50] [39_37433] d_loss: 0.08057, g_loss: 1.821
[39/50] [39_37434] d_loss: 0.07194, g_loss: 1.727
[39/50] [39_37435] d_loss: 0.01507, g_loss: 2.314
[39/50] [39_37436] d_loss: 0.02901, g_loss: 2.825
[39/50] [39_37437] d_loss: 0.02417, g_loss: 3.253
[39/50] [39_37438] d_loss: 0.01494, g_loss: 2.945
[39/50] [39_37439] d_loss: 0.1617, g_loss: 0.6908
[39/50] [39_37440] d_loss: 0.3311, g_loss: 1.713
[39/50] [39_37441] d_loss: 0.06433, g_loss: 2.465
[39/50] [39_37442] d_loss: 0.00908, g_loss: 2.657
[39/50] [39_37443] d_loss: 0.05169, g_loss: 2.643
[39/50] [39_37444] d_loss: 0.09409, g_loss: 1.826
[39/50] [39_37445] d_loss: 0.00019, g_loss: 1.818
[39/50] [39_37446] d_loss: 0.0006111, g_loss: 2.145
[39/50] [39_37447] d_loss: 0.006225, g_loss: 2.705
[39/50] [39_37448] d_loss: 0.0716, g_loss: 1.92
[39/50] [39_37449] d_loss: 0.01661, g_loss: 3.382
[39/5

##############
[-329.27134051 -278.23830336 -176.02547098 -129.24523994 -336.13402907
 -128.43120414 -185.66831904 -213.11229645 -249.70724153    8.131677  ]
##########
[40/50] [40_37601] d_loss: 0.03394, g_loss: 2.711
[40/50] [40_37602] d_loss: 0.0, g_loss: 2.724
[40/50] [40_37603] d_loss: 0.1095, g_loss: 1.984
[40/50] [40_37604] d_loss: 0.05381, g_loss: 2.646
[40/50] [40_37605] d_loss: 0.03956, g_loss: 1.917
[40/50] [40_37606] d_loss: 0.05434, g_loss: 2.592
[40/50] [40_37607] d_loss: 0.03492, g_loss: 2.388
[40/50] [40_37608] d_loss: 0.07229, g_loss: 1.608
[40/50] [40_37609] d_loss: 0.07063, g_loss: 1.134
[40/50] [40_37610] d_loss: 0.05035, g_loss: 2.306
[40/50] [40_37611] d_loss: 0.001063, g_loss: 2.371
[40/50] [40_37612] d_loss: 0.003583, g_loss: 2.366
[40/50] [40_37613] d_loss: 0.06045, g_loss: 3.078
[40/50] [40_37614] d_loss: 0.003475, g_loss: 2.503
[40/50] [40_37615] d_loss: 0.03861, g_loss: 2.156
[40/50] [40_37616] d_loss: 0.1298, g_loss: 3.922
[40/50] [40_37617] d_loss: 0.0, g_

[40/50] [40_37771] d_loss: 0.05942, g_loss: 2.435
[40/50] [40_37772] d_loss: 0.0177, g_loss: 2.772
[40/50] [40_37773] d_loss: 0.0366, g_loss: 2.573
[40/50] [40_37774] d_loss: 0.0, g_loss: 2.275
[40/50] [40_37775] d_loss: 0.01441, g_loss: 3.116
[40/50] [40_37776] d_loss: 0.05319, g_loss: 2.54
[40/50] [40_37777] d_loss: 0.02187, g_loss: 1.727
[40/50] [40_37778] d_loss: 0.1016, g_loss: 1.937
[40/50] [40_37779] d_loss: 0.1616, g_loss: 1.719
[40/50] [40_37780] d_loss: 0.08453, g_loss: 1.217
[40/50] [40_37781] d_loss: 0.1794, g_loss: 1.81
[40/50] [40_37782] d_loss: 0.1011, g_loss: 2.675
[40/50] [40_37783] d_loss: 0.01438, g_loss: 2.398
[40/50] [40_37784] d_loss: 0.1224, g_loss: 1.644
[40/50] [40_37785] d_loss: 0.06858, g_loss: 2.277
[40/50] [40_37786] d_loss: 0.00491, g_loss: 2.585
[40/50] [40_37787] d_loss: 0.1074, g_loss: 2.235
[40/50] [40_37788] d_loss: 0.05899, g_loss: 1.875
[40/50] [40_37789] d_loss: 0.0973, g_loss: 1.209
[40/50] [40_37790] d_loss: 0.1974, g_loss: 1.707
[40/50] [40_3779

[40/50] [40_37942] d_loss: 0.1557, g_loss: 2.256
[40/50] [40_37943] d_loss: 0.1138, g_loss: 2.277
[40/50] [40_37944] d_loss: 0.04172, g_loss: 2.36
[40/50] [40_37945] d_loss: 0.02082, g_loss: 3.774
[40/50] [40_37946] d_loss: 0.06794, g_loss: 3.263
[40/50] [40_37947] d_loss: 0.01036, g_loss: 2.886
[40/50] [40_37948] d_loss: 0.06883, g_loss: 4.076
[40/50] [40_37949] d_loss: 0.2244, g_loss: 2.206
[40/50] [40_37950] d_loss: 0.02961, g_loss: 2.075
[40/50] [40_37951] d_loss: 0.04432, g_loss: 2.284
[40/50] [40_37952] d_loss: 0.03198, g_loss: 2.615
[40/50] [40_37953] d_loss: 0.0469, g_loss: 3.992
[40/50] [40_37954] d_loss: 0.05313, g_loss: 4.531
[40/50] [40_37955] d_loss: 0.1691, g_loss: 1.724
[40/50] [40_37956] d_loss: 0.04081, g_loss: 1.908
[40/50] [40_37957] d_loss: 0.1707, g_loss: 1.491
[40/50] [40_37958] d_loss: 0.08292, g_loss: 2.348
[40/50] [40_37959] d_loss: 0.1107, g_loss: 1.872
[40/50] [40_37960] d_loss: 0.06024, g_loss: 2.729
[40/50] [40_37961] d_loss: 0.08282, g_loss: 1.851
[40/50] 